# Predicting Emotions in Twitter Sentiment Analysis part 4 - New Starbucks data

##  1. Extract tweets from Twitter API

In [1]:
import tweepy
import json
import pprint
import nltk
import spacy
import pandas as pd
import matplotlib.pyplot as plt
import tweepy as tw
import regex as re

from textblob import TextBlob
from TwitterAPI import TwitterAPI
from langdetect import detect

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *

from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from autocorrect import Speller

In [ ]:
path_auth = 'auth_twitter.json'
auth = json.loads(open(path_auth).read())
pp = pprint.PrettyPrinter(indent=4)
# For debugging only:
pp.pprint(auth)

In [6]:
client = tweepy.Client(
    bearer_token=my_bearer_token,
    consumer_key=my_consumer_key,
    consumer_secret=my_consumer_secret,
    access_token=my_access_token,
    access_token_secret=my_access_token_secret
)

In [28]:
api = TwitterAPI(consumer_key=my_consumer_key,
                 consumer_secret=my_consumer_secret,
                 access_token_key=my_access_token,
                 access_token_secret=my_access_token_secret)

In [29]:
pwd()

'C:\\Users\\katel\\OneDrive\\Desktop\\Data Science IOD Course\\Projects'

In [43]:
query = ['Starbucks']

tweets = client.search_recent_tweets(query=query, tweet_fields=['context_annotations', 'created_at'],
                                     media_fields=['preview_image_url'],
                                     max_results=100)

In [7]:
# OR trial this 
# auth = tweepy.OAuthHandler(my_consumer_key, my_consumer_secret)
# auth.set_access_token(my_access_token, my_access_token_secret)
# api = tweepy.API(auth)

In [8]:
# query = ['Starbucks']

#tweets = tweepy.Cursor(api.search_tweets, q=query, tweet_mode='extended', 
                      lang='en').items(100)

#for tweet in tweets:
#    print(tweet.full_text, '\n')

RT @ohgiisellee: i hope everytime he sees a starbucks he thinks of me🫣 

Made an extremely yummy Starbucks copycat oat bar and I am very pleased! Definitely make these as a quick breakfast option more often 😸 https://t.co/q9wdtxyPB2 

@DaitoYoshi Starbucks doing well 

RT @meeloodyya: “no we can’t go to starbucks” https://t.co/wDswaTstw8 

RT @Abramjee: This woman was verbally assaulted and kicked by a man at Starbucks East Gate Mall. The attacker said she was sitting for “too… 

im tweakin or is this the video of shorty that was naked w the fake starbucks https://t.co/9mHneGqiWA 

RT @MarcLobliner: This might be my new favorite shirt - @tigerfitnesscom Starbucks style!

Should have it up tomorrow while supplies last a… 

RT @meeloodyya: “no we can’t go to starbucks” https://t.co/wDswaTstw8 

RT @meeloodyya: “no we can’t go to starbucks” https://t.co/wDswaTstw8 

RT @KhoaPhan96: Cute starbucks staff #ぺこらーと #SticcArtBro
This is for you @JQHNHARQLD , happy birthday https://t.co/Z3UqMQA8w

RT @meeloodyya: “no we can’t go to starbucks” https://t.co/wDswaTstw8 

Free Starbucks drink cause it’s my birthday 🥰🫶🏻 

RT @PrezLives2022: An anti lesbian Karen goes viral for saying lesbians touch children and rape children… these haters have lost all sense… 

RT @SBWorkersUnited: Emotionally, spring starts the first time you can organize a union without Starbucks union-busting. 

@typesfast Starbucks on Main Street no longer has a soy milk option is my guess 

RT @meeloodyya: “no we can’t go to starbucks” https://t.co/wDswaTstw8 

RT @xxoorita: Starbucks employee looked so sad i literally bursted out in tears when I went back to my car cause i could tell she was going… 

RT @meeloodyya: “no we can’t go to starbucks” https://t.co/wDswaTstw8 

Check out this listing I just added to my #Poshmark closet: ⚡️FLASH SALE⚡️ Starbucks Blue ombré cold cup key chain 2023. https://t.co/S8oChovTq8 #shopmycloset @poshmarkapp 

RT @meeloodyya: “no we can’t go to starbucks” https://t.co/wDswaTstw8 

In [35]:
tweets.data[2].data

{'edit_history_tweet_ids': ['1633793575593844737'],
 'text': '@HalHaneyArt Somebody probably told him that Starbucks had a drive thru, and he misunderstood.',
 'context_annotations': [{'domain': {'id': '45',
    'name': 'Brand Vertical',
    'description': 'Top level entities that describe a Brands industry'},
   'entity': {'id': '781974596161376261', 'name': 'Dining'}},
  {'domain': {'id': '30',
    'name': 'Entities [Entity Service]',
    'description': 'Entity Service top level domain, every item that is in Entity Service should be in this domain'},
   'entity': {'id': '781974597017083904', 'name': 'QSR - Dining'}},
  {'domain': {'id': '46',
    'name': 'Business Taxonomy',
    'description': 'Categories within Brand Verticals that narrow down the scope of Brands'},
   'entity': {'id': '1557696888568045569',
    'name': 'Food & Beverage Business',
    'description': 'Brands, companies, advertisers and every non-person handle with the profit intent and related to foods & beverages su

In [36]:
tweet_list = []
for tweet in tweets.data:
    tweet_list.append(tweet.data)
df = pd.DataFrame(tweet_list)
df['created_at'] = pd.to_datetime(df['created_at'])
df.set_index('created_at', inplace=True)

In [37]:
df.reset_index(inplace=True)

In [38]:
df = df[['text']]
df

,text
0,RT @eye_gnats: Hooters Lucina guy also has a c...
1,you fumbled so hard @Starbucks \nhire me pleas...
2,@HalHaneyArt Somebody probably told him that S...
3,"RT @LanaGallery: “She juggles a red vape, the ..."
4,RT @GoldretrieverUS: OH MOM- You shouldn't hav...
...,...
95,RT @MissJadeJewels: Skip your Starbucks ☕️ Tod...
96,@JAlexBenham @MrBeast Because not everyone out...
97,I’m having my starbucks mocha frappuccino with...
98,スタバ\n\n新型コロナウイルス感染症に関わる弊社安全対策について（3/7更新） | スター...


In [40]:
# drop duplicated tweets
df.drop_duplicates(subset='text', inplace=True)

## 2. Preprocess tweets 

In [48]:
# clean tweet
def clean_tweet(tweet):
    tweet = re.sub('@([a-zA-Z0-9_]+)', '', tweet)
    tweet = re.sub('#([a-zA-Z0-9_]+)', '', tweet)
    tweet = re.sub('http:\S+', '', tweet)
    tweet = re.sub('https:\S+', '', tweet)
    tweet = re.sub('[^a-zA-Z]', ' ', tweet) # remove non word characters
    tweet = re.sub('[0-9]+', '', tweet) # remove words and digits containing digits
    tweet = re.sub('\s\s+', ' ', tweet)
    tweet = re.sub('RT', '', tweet)
    #tweet = re.sub('', '', tweet)
    
    return tweet

In [49]:
%%time
# clean tweets
df['text_clean'] = df['text'].apply(clean_tweet)

Wall time: 56.1 ms


In [50]:
df

,text,text_clean
0,RT @eye_gnats: Hooters Lucina guy also has a c...,Hooters Lucina guy also has a commission of S...
1,you fumbled so hard @Starbucks \nhire me pleas...,you fumbled so hard hire me please hire me
2,@HalHaneyArt Somebody probably told him that S...,Somebody probably told him that Starbucks had...
3,"RT @LanaGallery: “She juggles a red vape, the ...",She juggles a red vape the keys to her truck ...
4,RT @GoldretrieverUS: OH MOM- You shouldn't hav...,OH MOM You shouldn t have for me And dinner l...
...,...,...
94,RT @mndsss: สั่ง starbucks strawberry acai wit...,starbucks strawberry acai with lemonade espre...
96,@JAlexBenham @MrBeast Because not everyone out...,Because not everyone out of range of your loc...
97,I’m having my starbucks mocha frappuccino with...,I m having my starbucks mocha frappuccino with...
98,スタバ\n\n新型コロナウイルス感染症に関わる弊社安全対策について（3/7更新） | スター...,


In [51]:
df['text_clean'] = df['text_clean'].str.lower()

In [52]:
# spelling correction using spellchecker
spell = Speller()

def spell_autocorrect(text):
    correct_spell_words = []
    # initialize Speller object for english language with 'en'
    spell_corrector = Speller(lang='en')
    
    for word in word_tokenize(text):
        # correct spell word
        correct_word = spell_corrector(word)
        correct_spell_words.append(correct_word)

    correct_spelling = ' '.join(correct_spell_words)
    return correct_spelling

In [53]:
df['text_spell'] = df['text_clean'].apply(lambda row: spell_autocorrect(row))

In [54]:
df

,text,text_clean,text_spell
0,RT @eye_gnats: Hooters Lucina guy also has a c...,hooters lucina guy also has a commission of s...,shooters lucida guy also has a commission of s...
1,you fumbled so hard @Starbucks \nhire me pleas...,you fumbled so hard hire me please hire me,you fumbled so hard hire me please hire me
2,@HalHaneyArt Somebody probably told him that S...,somebody probably told him that starbucks had...,somebody probably told him that starbucks had ...
3,"RT @LanaGallery: “She juggles a red vape, the ...",she juggles a red vape the keys to her truck ...,she jungles a red tape the keys to her truck a...
4,RT @GoldretrieverUS: OH MOM- You shouldn't hav...,oh mom you shouldn t have for me and dinner l...,oh mom you shouldn t have for me and dinner lo...
...,...,...,...
94,RT @mndsss: สั่ง starbucks strawberry acai wit...,starbucks strawberry acai with lemonade espre...,starbucks strawberry aca with lemonade espress...
96,@JAlexBenham @MrBeast Because not everyone out...,because not everyone out of range of your loc...,because not everyone out of range of your loca...
97,I’m having my starbucks mocha frappuccino with...,i m having my starbucks mocha frappuccino with...,i m having my starbucks mecha frappuccino with...
98,スタバ\n\n新型コロナウイルス感染症に関わる弊社安全対策について（3/7更新） | スター...,,


In [55]:
# make a list of english stopwords
stopwords = nltk.corpus.stopwords.words('english')
# extend list with custom stopwords
# my_stopwords = ['']
# stopwords.extend(my_stopwords)

In [56]:
df['text_token'] = df.apply(lambda row: word_tokenize(row['text_spell']), axis=1)
df['text_token']= df['text_token'].apply(lambda x: [item for item in x if item not in stopwords])

In [57]:
stemmer = nltk.SnowballStemmer("english")

def stem_tweet(df):
    stem_text = ' '.join([stemmer.stem(tweet) for tweet in df])
    return stem_text

df['text_stem']= df['text_token'].apply(lambda x: stem_tweet(x))

In [58]:
df

,text,text_clean,text_spell,text_token,text_stem
0,RT @eye_gnats: Hooters Lucina guy also has a c...,hooters lucina guy also has a commission of s...,shooters lucida guy also has a commission of s...,"[shooters, lucida, guy, also, commission, star...",shooter lucida guy also commiss starbuck reina...
1,you fumbled so hard @Starbucks \nhire me pleas...,you fumbled so hard hire me please hire me,you fumbled so hard hire me please hire me,"[fumbled, hard, hire, please, hire]",fumbl hard hire pleas hire
2,@HalHaneyArt Somebody probably told him that S...,somebody probably told him that starbucks had...,somebody probably told him that starbucks had ...,"[somebody, probably, told, starbucks, drive, t...",somebodi probabl told starbuck drive thru misu...
3,"RT @LanaGallery: “She juggles a red vape, the ...",she juggles a red vape the keys to her truck ...,she jungles a red tape the keys to her truck a...,"[jungles, red, tape, keys, truck, vent, starbu...",jungl red tape key truck vent starbuck cup iph...
4,RT @GoldretrieverUS: OH MOM- You shouldn't hav...,oh mom you shouldn t have for me and dinner l...,oh mom you shouldn t have for me and dinner lo...,"[oh, mom, dinner, looks, good, wait, getting]",oh mom dinner look good wait get
...,...,...,...,...,...
94,RT @mndsss: สั่ง starbucks strawberry acai wit...,starbucks strawberry acai with lemonade espre...,starbucks strawberry aca with lemonade espress...,"[starbucks, strawberry, aca, lemonade, espress...",starbuck strawberri aca lemonad espresso shot
96,@JAlexBenham @MrBeast Because not everyone out...,because not everyone out of range of your loc...,because not everyone out of range of your loca...,"[everyone, range, local, starbucks, starving, ...",everyon rang local starbuck starv mayb thing s...
97,I’m having my starbucks mocha frappuccino with...,i m having my starbucks mocha frappuccino with...,i m having my starbucks mecha frappuccino with...,"[starbucks, mecha, frappuccino, chobani, vanil...",starbuck mecha frappuccino chobani vanilla gre...
98,スタバ\n\n新型コロナウイルス感染症に関わる弊社安全対策について（3/7更新） | スター...,,,[],


In [59]:
# remove short words less than 2 characters
df['text_pre'] = df['text_stem'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

In [60]:
df

,text,text_clean,text_spell,text_token,text_stem,text_pre
0,RT @eye_gnats: Hooters Lucina guy also has a c...,hooters lucina guy also has a commission of s...,shooters lucida guy also has a commission of s...,"[shooters, lucida, guy, also, commission, star...",shooter lucida guy also commiss starbuck reina...,shooter lucida guy also commiss starbuck reina...
1,you fumbled so hard @Starbucks \nhire me pleas...,you fumbled so hard hire me please hire me,you fumbled so hard hire me please hire me,"[fumbled, hard, hire, please, hire]",fumbl hard hire pleas hire,fumbl hard hire pleas hire
2,@HalHaneyArt Somebody probably told him that S...,somebody probably told him that starbucks had...,somebody probably told him that starbucks had ...,"[somebody, probably, told, starbucks, drive, t...",somebodi probabl told starbuck drive thru misu...,somebodi probabl told starbuck drive thru misu...
3,"RT @LanaGallery: “She juggles a red vape, the ...",she juggles a red vape the keys to her truck ...,she jungles a red tape the keys to her truck a...,"[jungles, red, tape, keys, truck, vent, starbu...",jungl red tape key truck vent starbuck cup iph...,jungl red tape key truck vent starbuck cup iph...
4,RT @GoldretrieverUS: OH MOM- You shouldn't hav...,oh mom you shouldn t have for me and dinner l...,oh mom you shouldn t have for me and dinner lo...,"[oh, mom, dinner, looks, good, wait, getting]",oh mom dinner look good wait get,mom dinner look good wait get
...,...,...,...,...,...,...
94,RT @mndsss: สั่ง starbucks strawberry acai wit...,starbucks strawberry acai with lemonade espre...,starbucks strawberry aca with lemonade espress...,"[starbucks, strawberry, aca, lemonade, espress...",starbuck strawberri aca lemonad espresso shot,starbuck strawberri aca lemonad espresso shot
96,@JAlexBenham @MrBeast Because not everyone out...,because not everyone out of range of your loc...,because not everyone out of range of your loca...,"[everyone, range, local, starbucks, starving, ...",everyon rang local starbuck starv mayb thing s...,everyon rang local starbuck starv mayb thing s...
97,I’m having my starbucks mocha frappuccino with...,i m having my starbucks mocha frappuccino with...,i m having my starbucks mecha frappuccino with...,"[starbucks, mecha, frappuccino, chobani, vanil...",starbuck mecha frappuccino chobani vanilla gre...,starbuck mecha frappuccino chobani vanilla gre...
98,スタバ\n\n新型コロナウイルス感染症に関わる弊社安全対策について（3/7更新） | スター...,,,[],,


In [61]:
# join words together 
df['text_str'] = df['text_token'].apply(lambda x: ' '.join([item for item in x]))

In [62]:
# after all processing, create a list of all words
all_words = ' '.join([word for word in df['text_str']])

In [63]:
df

,text,text_clean,text_spell,text_token,text_stem,text_pre,text_str
0,RT @eye_gnats: Hooters Lucina guy also has a c...,hooters lucina guy also has a commission of s...,shooters lucida guy also has a commission of s...,"[shooters, lucida, guy, also, commission, star...",shooter lucida guy also commiss starbuck reina...,shooter lucida guy also commiss starbuck reina...,shooters lucida guy also commission starbucks ...
1,you fumbled so hard @Starbucks \nhire me pleas...,you fumbled so hard hire me please hire me,you fumbled so hard hire me please hire me,"[fumbled, hard, hire, please, hire]",fumbl hard hire pleas hire,fumbl hard hire pleas hire,fumbled hard hire please hire
2,@HalHaneyArt Somebody probably told him that S...,somebody probably told him that starbucks had...,somebody probably told him that starbucks had ...,"[somebody, probably, told, starbucks, drive, t...",somebodi probabl told starbuck drive thru misu...,somebodi probabl told starbuck drive thru misu...,somebody probably told starbucks drive thru mi...
3,"RT @LanaGallery: “She juggles a red vape, the ...",she juggles a red vape the keys to her truck ...,she jungles a red tape the keys to her truck a...,"[jungles, red, tape, keys, truck, vent, starbu...",jungl red tape key truck vent starbuck cup iph...,jungl red tape key truck vent starbuck cup iph...,jungles red tape keys truck vent starbucks cup...
4,RT @GoldretrieverUS: OH MOM- You shouldn't hav...,oh mom you shouldn t have for me and dinner l...,oh mom you shouldn t have for me and dinner lo...,"[oh, mom, dinner, looks, good, wait, getting]",oh mom dinner look good wait get,mom dinner look good wait get,oh mom dinner looks good wait getting
...,...,...,...,...,...,...,...
94,RT @mndsss: สั่ง starbucks strawberry acai wit...,starbucks strawberry acai with lemonade espre...,starbucks strawberry aca with lemonade espress...,"[starbucks, strawberry, aca, lemonade, espress...",starbuck strawberri aca lemonad espresso shot,starbuck strawberri aca lemonad espresso shot,starbucks strawberry aca lemonade espresso shot
96,@JAlexBenham @MrBeast Because not everyone out...,because not everyone out of range of your loc...,because not everyone out of range of your loca...,"[everyone, range, local, starbucks, starving, ...",everyon rang local starbuck starv mayb thing s...,everyon rang local starbuck starv mayb thing s...,everyone range local starbucks starving maybe ...
97,I’m having my starbucks mocha frappuccino with...,i m having my starbucks mocha frappuccino with...,i m having my starbucks mecha frappuccino with...,"[starbucks, mecha, frappuccino, chobani, vanil...",starbuck mecha frappuccino chobani vanilla gre...,starbuck mecha frappuccino chobani vanilla gre...,starbucks mecha frappuccino chobani vanilla gr...
98,スタバ\n\n新型コロナウイルス感染症に関わる弊社安全対策について（3/7更新） | スター...,,,[],,,


In [64]:
# filter for words that are finalised and processed in the 2nd last col
df1= df[df['text_pre'].notnull()]

In [10]:
df1

,text,text_clean,text_spell,text_token,text_stem,text_pre,text_str
0,RT @eye_gnats: Hooters Lucina guy also has a c...,hooters lucina guy also has a commission of s...,shooters lucida guy also has a commission of s...,"['shooters', 'lucida', 'guy', 'also', 'commiss...",shooter lucida guy also commiss starbuck reina...,shooter lucida guy also commiss starbuck reina...,shooters lucida guy also commission starbucks ...
1,you fumbled so hard @Starbucks \nhire me pleas...,you fumbled so hard hire me please hire me,you fumbled so hard hire me please hire me,"['fumbled', 'hard', 'hire', 'please', 'hire']",fumbl hard hire pleas hire,fumbl hard hire pleas hire,fumbled hard hire please hire
2,@HalHaneyArt Somebody probably told him that S...,somebody probably told him that starbucks had...,somebody probably told him that starbucks had ...,"['somebody', 'probably', 'told', 'starbucks', ...",somebodi probabl told starbuck drive thru misu...,somebodi probabl told starbuck drive thru misu...,somebody probably told starbucks drive thru mi...
3,"RT @LanaGallery: “She juggles a red vape, the ...",she juggles a red vape the keys to her truck ...,she jungles a red tape the keys to her truck a...,"['jungles', 'red', 'tape', 'keys', 'truck', 'v...",jungl red tape key truck vent starbuck cup iph...,jungl red tape key truck vent starbuck cup iph...,jungles red tape keys truck vent starbucks cup...
4,RT @GoldretrieverUS: OH MOM- You shouldn't hav...,oh mom you shouldn t have for me and dinner l...,oh mom you shouldn t have for me and dinner lo...,"['oh', 'mom', 'dinner', 'looks', 'good', 'wait...",oh mom dinner look good wait get,mom dinner look good wait get,oh mom dinner looks good wait getting
...,...,...,...,...,...,...,...
76,RT @mndsss: สั่ง starbucks strawberry acai wit...,starbucks strawberry acai with lemonade espre...,starbucks strawberry aca with lemonade espress...,"['starbucks', 'strawberry', 'aca', 'lemonade',...",starbuck strawberri aca lemonad espresso shot,starbuck strawberri aca lemonad espresso shot,starbucks strawberry aca lemonade espresso shot
77,@JAlexBenham @MrBeast Because not everyone out...,because not everyone out of range of your loc...,because not everyone out of range of your loca...,"['everyone', 'range', 'local', 'starbucks', 's...",everyon rang local starbuck starv mayb thing s...,everyon rang local starbuck starv mayb thing s...,everyone range local starbucks starving maybe ...
78,I’m having my starbucks mocha frappuccino with...,i m having my starbucks mocha frappuccino with...,i m having my starbucks mecha frappuccino with...,"['starbucks', 'mecha', 'frappuccino', 'chobani...",starbuck mecha frappuccino chobani vanilla gre...,starbuck mecha frappuccino chobani vanilla gre...,starbucks mecha frappuccino chobani vanilla gr...
79,スタバ\n\n新型コロナウイルス感染症に関わる弊社安全対策について（3/7更新） | スター...,,NaN,[],NaN,NaN,NaN


In [41]:
df1['text_str'] = df1['text_str'].astype(str)

In [43]:
# new df contains only ones with english 
df_new = df1[df1.text_str.apply(detect).eq('en')]
df_new

,text,text_clean,text_spell,text_token,text_stem,text_pre,text_str
0,RT @eye_gnats: Hooters Lucina guy also has a c...,hooters lucina guy also has a commission of s...,shooters lucida guy also has a commission of s...,"['shooters', 'lucida', 'guy', 'also', 'commiss...",shooter lucida guy also commiss starbuck reina...,shooter lucida guy also commiss starbuck reina...,shooters lucida guy also commission starbucks ...
1,you fumbled so hard @Starbucks \nhire me pleas...,you fumbled so hard hire me please hire me,you fumbled so hard hire me please hire me,"['fumbled', 'hard', 'hire', 'please', 'hire']",fumbl hard hire pleas hire,fumbl hard hire pleas hire,fumbled hard hire please hire
2,@HalHaneyArt Somebody probably told him that S...,somebody probably told him that starbucks had...,somebody probably told him that starbucks had ...,"['somebody', 'probably', 'told', 'starbucks', ...",somebodi probabl told starbuck drive thru misu...,somebodi probabl told starbuck drive thru misu...,somebody probably told starbucks drive thru mi...
3,"RT @LanaGallery: “She juggles a red vape, the ...",she juggles a red vape the keys to her truck ...,she jungles a red tape the keys to her truck a...,"['jungles', 'red', 'tape', 'keys', 'truck', 'v...",jungl red tape key truck vent starbuck cup iph...,jungl red tape key truck vent starbuck cup iph...,jungles red tape keys truck vent starbucks cup...
5,RT @cryptolab119: CryptoLab x DalGomz\n\n5 x S...,cryptolab x dalgomz x starbucks or trx airdro...,cryptolab x dalgomz x starbucks or try airdrop...,"['cryptolab', 'x', 'dalgomz', 'x', 'starbucks'...",cryptolab x dalgomz x starbuck tri airdrop eve...,cryptolab dalgomz starbuck tri airdrop event c...,cryptolab x dalgomz x starbucks try airdrop ev...
6,RT @SBWorkersUnited: Now this is y’allidarity!...,now this is y allidarity partners at the hwy ...,now this is y solidarity partners at the hwy a...,"['solidarity', 'partners', 'hwy', 'amp', 'brin...",solidar partner hwi amp brinkley hill starbuck...,solidar partner hwi amp brinkley hill starbuck...,solidarity partners hwy amp brinkley hill star...
7,【譲渡】Starbucks My Customize Journey Set\n\n落選した...,starbucks my customize journey set,starbucks my customize journey set,"['starbucks', 'customize', 'journey', 'set']",starbuck custom journey set,starbuck custom journey set,starbucks customize journey set
8,RT @MissJadeJewels: Skip your Starbucks ☕️ Tod...,skip your starbucks today and join me instead...,skip your starbucks today and join me instead ...,"['skip', 'starbucks', 'today', 'join', 'instea...",skip starbuck today join instead give dick love,skip starbuck today join instead give dick love,skip starbucks today join instead give dick love
12,"Nervous to apply for a job like ""Barista - Sto...",nervous to apply for a job like barista store ...,nervous to apply for a job like barnstar store...,"['nervous', 'apply', 'job', 'like', 'barnstar'...",nervous appli job like barnstar store glasgow ...,nervous appli job like barnstar store glasgow ...,nervous apply job like barnstar store glasgow ...
13,if this is true di ako ready umiyak sa starbuc...,if this is true di ako ready umiyak sa starbuc...,if this is true di ako ready miya sa starbucks...,"['true', 'di', 'ako', 'ready', 'miya', 'sa', '...",true di ako readi miya sa starbuck ng med citi,true ako readi miya starbuck med citi,true di ako ready miya sa starbucks ng med city


In [ ]:
df_new.to_csv('starbuckstweets.csv', index=False)

In [3]:
df_new = pd.read_csv('starbuckstweets.csv')

In [4]:
df_new.info()
df_new.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        43 non-null     object
 1   text_clean  43 non-null     object
 2   text_spell  43 non-null     object
 3   text_token  43 non-null     object
 4   text_stem   43 non-null     object
 5   text_pre    43 non-null     object
 6   text_str    43 non-null     object
dtypes: object(7)
memory usage: 2.5+ KB


text          0
text_clean    0
text_spell    0
text_token    0
text_stem     0
text_pre      0
text_str      0
dtype: int64

### 3. Label tweets 

In [19]:
sia = SentimentIntensityAnalyzer()

In [9]:
df_new["text_str"].astype(str)

0     shooters lucida guy also commission starbucks ...
1                         fumbled hard hire please hire
2     somebody probably told starbucks drive thru mi...
3     jungles red tape keys truck vent starbucks cup...
4     cryptolab x dalgomz x starbucks try airdrop ev...
5     solidarity partners hwy amp brinkley hill star...
6                       starbucks customize journey set
7      skip starbucks today join instead give dick love
8     nervous apply job like barnstar store glasgow ...
9       true di ako ready miya sa starbucks ng med city
10    national women day deserve starbucks flowers t...
11                    literally thought starbucks drink
12    drink get starbucks ion want nothing sweet hig...
13    said yes went way later day stopped starbucks ...
14    pizza amp starbucks coffee get day going grill...
15                                     starbucks coffee
16           anybody like starbucks know nothing coffee
17    breaking jury awards million black mother 

In [17]:
sb = df_new[['text_str']]

In [20]:
# Applying Model, Variable Creation
sb['sentiment_score'] = sb['text_str'].apply(lambda text: sia.polarity_scores(text)['compound'])

C:\Users\katel\AppData\Local\Temp\ipykernel_11364\3367839023.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sb['sentiment_score'] = sb['text_str'].apply(lambda text: sia.polarity_scores(text)['compound'])


In [22]:
sb.groupby('sentiment_score').count()

,text_str
sentiment_score,
-0.8492,1
-0.7351,1
-0.6908,1
-0.4404,1
-0.3400,2
-0.2914,1
0.0000,15
0.0258,1
0.0880,1


In [23]:
# assign sentiment, have to assign neutral as 31 scores are 0 
sb['Sentiment']=''
sb.loc[sb['sentiment_score']>0, 'Sentiment'] = 'Positive'
sb.loc[sb['sentiment_score'] == 0, 'Sentiment'] = 'Neutral'
sb.loc[sb['sentiment_score']<0, 'Sentiment'] = 'Negative'

C:\Users\katel\AppData\Local\Temp\ipykernel_11364\3019681268.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sb['Sentiment']=''


In [24]:
# label sentiment 
sb['SIA_label']=''
sb.loc[sb['Sentiment']=='Positive', 'SIA_label'] = '1'
sb.loc[sb['Sentiment']=='Neutral', 'SIA_label'] = '0'
sb.loc[sb['Sentiment']=='Negative', 'SIA_label'] = '-1'

C:\Users\katel\AppData\Local\Temp\ipykernel_11364\2732649384.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sb['SIA_label']=''


In [28]:
sb.to_csv('sblabelled.csv', index=False)

In [2]:
pd.read_csv('sblabelled.csv')

,text_str,sentiment_score,Sentiment,SIA_label
0,shooters lucida guy also commission starbucks ...,-0.4404,Negative,-1
1,fumbled hard hire please hire,0.2263,Positive,1
2,somebody probably told starbucks drive thru mi...,-0.3400,Negative,-1
3,jungles red tape keys truck vent starbucks cup...,0.0000,Neutral,0
4,cryptolab x dalgomz x starbucks try airdrop ev...,0.0000,Neutral,0
5,solidarity partners hwy amp brinkley hill star...,0.2960,Positive,1
6,starbucks customize journey set,0.0000,Neutral,0
7,skip starbucks today join instead give dick love,0.4767,Positive,1
8,nervous apply job like barnstar store glasgow ...,0.4215,Positive,1
9,true di ako ready miya sa starbucks ng med city,0.6486,Positive,1


In [6]:
check = pd.read_csv('../projects/starbuckstweets.csv')

In [7]:
check

,text,text_clean,text_spell,text_token,text_stem,text_pre,text_str
0,RT @eye_gnats: Hooters Lucina guy also has a c...,hooters lucina guy also has a commission of s...,shooters lucida guy also has a commission of s...,"['shooters', 'lucida', 'guy', 'also', 'commiss...",shooter lucida guy also commiss starbuck reina...,shooter lucida guy also commiss starbuck reina...,shooters lucida guy also commission starbucks ...
1,you fumbled so hard @Starbucks \nhire me pleas...,you fumbled so hard hire me please hire me,you fumbled so hard hire me please hire me,"['fumbled', 'hard', 'hire', 'please', 'hire']",fumbl hard hire pleas hire,fumbl hard hire pleas hire,fumbled hard hire please hire
2,@HalHaneyArt Somebody probably told him that S...,somebody probably told him that starbucks had...,somebody probably told him that starbucks had ...,"['somebody', 'probably', 'told', 'starbucks', ...",somebodi probabl told starbuck drive thru misu...,somebodi probabl told starbuck drive thru misu...,somebody probably told starbucks drive thru mi...
3,"RT @LanaGallery: “She juggles a red vape, the ...",she juggles a red vape the keys to her truck ...,she jungles a red tape the keys to her truck a...,"['jungles', 'red', 'tape', 'keys', 'truck', 'v...",jungl red tape key truck vent starbuck cup iph...,jungl red tape key truck vent starbuck cup iph...,jungles red tape keys truck vent starbucks cup...
4,RT @cryptolab119: CryptoLab x DalGomz\n\n5 x S...,cryptolab x dalgomz x starbucks or trx airdro...,cryptolab x dalgomz x starbucks or try airdrop...,"['cryptolab', 'x', 'dalgomz', 'x', 'starbucks'...",cryptolab x dalgomz x starbuck tri airdrop eve...,cryptolab dalgomz starbuck tri airdrop event c...,cryptolab x dalgomz x starbucks try airdrop ev...
5,RT @SBWorkersUnited: Now this is y’allidarity!...,now this is y allidarity partners at the hwy ...,now this is y solidarity partners at the hwy a...,"['solidarity', 'partners', 'hwy', 'amp', 'brin...",solidar partner hwi amp brinkley hill starbuck...,solidar partner hwi amp brinkley hill starbuck...,solidarity partners hwy amp brinkley hill star...
6,【譲渡】Starbucks My Customize Journey Set\n\n落選した...,starbucks my customize journey set,starbucks my customize journey set,"['starbucks', 'customize', 'journey', 'set']",starbuck custom journey set,starbuck custom journey set,starbucks customize journey set
7,RT @MissJadeJewels: Skip your Starbucks ☕️ Tod...,skip your starbucks today and join me instead...,skip your starbucks today and join me instead ...,"['skip', 'starbucks', 'today', 'join', 'instea...",skip starbuck today join instead give dick love,skip starbuck today join instead give dick love,skip starbucks today join instead give dick love
8,"Nervous to apply for a job like ""Barista - Sto...",nervous to apply for a job like barista store ...,nervous to apply for a job like barnstar store...,"['nervous', 'apply', 'job', 'like', 'barnstar'...",nervous appli job like barnstar store glasgow ...,nervous appli job like barnstar store glasgow ...,nervous apply job like barnstar store glasgow ...
9,if this is true di ako ready umiyak sa starbuc...,if this is true di ako ready umiyak sa starbuc...,if this is true di ako ready miya sa starbucks...,"['true', 'di', 'ako', 'ready', 'miya', 'sa', '...",true di ako readi miya sa starbuck ng med citi,true ako readi miya starbuck med citi,true di ako ready miya sa starbucks ng med city


In [23]:
pd.options.display.max_colwidth = 1000

Check full texts of unprocessed and processed text for comparison.

In [25]:
for column in check:
    print(check['text'])

0                                                                                                                                                                     RT @eye_gnats: Hooters Lucina guy also has a commission of Starbucks Reina and let me tell you. I’m not against the idea of tired milf bari…
1                                                                                                                                                                                                                                  you fumbled so hard @Starbucks \nhire me please hire me https://t.co/Zi9Br85Jv6
2                                                                                                                                                                                                                   @HalHaneyArt Somebody probably told him that Starbucks had a drive thru, and he misunderstood.
3                                                                              